In [189]:
import os
import json
import time
import random

import multiprocessing as mp


import io
import os.path
import re
import tarfile
import sys
import gensim
import pandas as pd
import numpy as np

from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from nltk import pos_tag


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm, naive_bayes, neural_network
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, make_scorer)

from nltk.tokenize import word_tokenize


In [2]:
repos_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

gutenberg_corpus_analysis_repo = os.path.join(repos_path, 'gutenberg_corpus_analysis')

In [3]:
gutenberg_repo_path = os.path.join(repos_path, 'gutenberg')
gutenberg_analysis_repo = os.path.join(repos_path, 'gutenberg-analysis')

src_dir = os.path.join(gutenberg_analysis_repo,'src')
sys.path.append(src_dir)
from data_io import get_book


gutenberg_src_dir = os.path.join(gutenberg_repo_path,'src')
sys.path.append(gutenberg_src_dir)

from metaquery import meta_query

sys.path.append(gutenberg_corpus_analysis_repo)
import misc_utils.dataset_filtering as dataset_filtering

In [ ]:
# Be able to reproduce results (won't work on chunking)
np.random.seed(500)

In [5]:
dataset='nikita_dataset'

In [6]:
train_csv = os.path.join(gutenberg_corpus_analysis_repo, dataset, 'final_train.csv')
test_csv = os.path.join(gutenberg_corpus_analysis_repo, dataset, 'final_test.csv')
val_csv = os.path.join(gutenberg_corpus_analysis_repo, dataset, 'final_val.csv')

pg_catalog_filepath=os.path.join(gutenberg_repo_path, 'metadata', 'pg_catalog.csv')

In [8]:
train_df = pd.read_csv(train_csv, index_col='Unnamed: 0')
test_df = pd.read_csv(test_csv, index_col='Unnamed: 0')
val_df = pd.read_csv(val_csv, index_col='Unnamed: 0')

train_df.head()

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects
2439,PG12810,"Uncle Sam's Boys with Pershing's Troops: Or, D...","Hancock, H. Irving (Harrie Irving)",1868.0,1922.0,['en'],78,"{'World War, 1914-1918 -- Juvenile fiction', '..."
2446,PG12819,"Dick Prescott's Second Year at West Point: Or,...","Hancock, H. Irving (Harrie Irving)",1868.0,1922.0,['en'],94,{'United States Military Academy -- Juvenile f...
25920,PG40605,"The Motor Boat Club at Nantucket; or, The Myst...","Hancock, H. Irving (Harrie Irving)",1868.0,1922.0,['en'],189,"{'Motorboats -- Juvenile fiction', 'Nantucket ..."
55435,PG8153,"The Young Engineers in Arizona; or, Laying Tra...","Hancock, H. Irving (Harrie Irving)",1868.0,1922.0,['en'],190,"{'Civil engineers -- Fiction', 'Arizona -- Fic..."
32899,PG48863,"The Motor Boat Club off Long Island; or, A Dar...","Hancock, H. Irving (Harrie Irving)",1868.0,1922.0,['en'],85,"{'Motorboats -- Juvenile fiction', 'Long Islan..."


In [9]:
len(train_df['author'].unique())

80

In [10]:
subj = train_df['subjects'].replace('set()',np.nan)
subj_docs = []
for h in subj:
    try:
        h = h.strip("{}")[1:-1]
    except AttributeError:
        subj_docs.append(h)
        continue
    h = h.replace(' -- ', '-')
    h = h.replace("', '","_")
    h = h.split('_')
    h = [item.replace(' ','').replace(',', ' ') for item in h]
    h = ' '.join(h)
    subj_docs.append(h)

In [11]:
train_df['subj_str']=subj_docs

In [12]:
#train_df['subject_str'] = train_df['subjects'].apply(lambda x: split_subjects(x))

In [13]:
#train_df = train_df.sample(500)

In [14]:
start = time.time()

train_df['text'] = train_df['id'].apply(lambda x: get_book(x, path_gutenberg=gutenberg_repo_path,level='text'))
test_df['text'] = test_df['id'].apply(lambda x: get_book(x, path_gutenberg=gutenberg_repo_path,level='text'))
val_df['text'] = val_df['id'].apply(lambda x: get_book(x, path_gutenberg=gutenberg_repo_path,level='text'))

end = time.time()
print(f'Time elapsed: {end-start} seconds')

Time elapsed: 5.267264366149902 seconds


In [170]:
# Define a function to apply the word, line and token counts
def enrich_dataframe(df):
    count_path = os.path.join(gutenberg_repo_path, 'data', 'counts')
    text_path = os.path.join(gutenberg_repo_path, 'data', 'text')
    token_path = os.path.join(gutenberg_repo_path, 'data', 'tokens')

    df['word_count'] = df['id'].apply(lambda pid: dataset_filtering.get_word_count(pid, count_path))
    df['unique_word_count'] = df['id'].apply(lambda pid: dataset_filtering.get_unique_word_count(pid, count_path))
    df['line_count'] = df['id'].apply(lambda pid: dataset_filtering.get_line_count(pid, text_path))
    df['token_count'] = df['id'].apply(lambda pid: dataset_filtering.get_token_count(pid, token_path))

    return df

In [ ]:
# train_df = enrich_dataframe(train_df)
# val_df = enrich_dataframe(tval_df)
# test_df = enrich_dataframe(test_df)

In [15]:
def skip_start_and_end(text):
    text = text.split(' ')
    text = text[50:-50]
    return ' '.join(text)

train_df['text'] = train_df['text'].apply(skip_start_and_end)
test_df['text'] = test_df['text'].apply(skip_start_and_end)
val_df['text'] = val_df['text'].apply(skip_start_and_end)


In [16]:
def make_random_chunks(text, num_chunks=10, chunk_size=1000, overlap=False):
    chunk = []
    words = text.split(' ')

    if num_chunks * chunk_size > len(words):
        return text
    for i in range(num_chunks):
        new_words = []
        num_words = len(words)
        if chunk_size > num_words:
            chunk = chunk + words
            words = []
            return ' '.join(chunk)
        start = random.randint(0, num_words)
        chunk = [*chunk,  *words[start:start+chunk_size]]
        #print(chunk)
        if start == 0:
            words = words[chunk_size:]
        elif start == num_words - chunk_size:
            words = words[0:start]
        else:
            words = words[0:start] + words[start+chunk_size:]
    return ' '.join(chunk)

In [17]:
train_df['chunks'] = train_df['text'].apply(lambda x: make_random_chunks(x, num_chunks=10, chunk_size = 1000, overlap=False))
test_df['chunks'] = test_df['text'].apply(lambda x: make_random_chunks(x, num_chunks=10, chunk_size = 1000, overlap=False))
val_df['chunks'] = val_df['text'].apply(lambda x: make_random_chunks(x, num_chunks=10, chunk_size = 1000, overlap=False))

In [23]:
start = time.time()
with mp.Pool(11) as pool:
    train_df['tokenized'] = pool.map(word_tokenize, train_df['chunks'])
end = time.time()
print(f'Took {end-start} seconds')


Took 12.728698253631592 seconds


In [24]:
start = time.time()
with mp.Pool(11) as pool:
    val_df['tokenized'] = pool.map(word_tokenize, val_df['chunks'])
end = time.time()
print(f'Took {end-start} seconds')



Took 2.776623249053955 seconds


In [25]:
start = time.time()
with mp.Pool(11) as pool:
    test_df['tokenized'] = pool.map(word_tokenize, test_df['chunks'])
end = time.time()
print(f'Took {end-start} seconds')



Took 3.5133683681488037 seconds


In [26]:
outfile=os.path.join(gutenberg_corpus_analysis_repo, 'tokenized', 'train_df_chunks_tokenized.pkl')
train_df.to_pickle(outfile)

outfile=os.path.join(gutenberg_corpus_analysis_repo, 'tokenized', 'val_df_chunks_tokenized.pkl')
val_df.to_pickle(outfile)

outfile=os.path.join(gutenberg_corpus_analysis_repo, 'tokenized', 'test_df_chunks_tokenized.pkl')
test_df.to_pickle(outfile)



In [ ]:
train_df = pd.read_pickle(outfile)

In [31]:
train_df[train_df['tokenized'].isnull()]

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,subj_str,text,chunks,tokenized


In [32]:
test_df[test_df['tokenized'].isnull()]

,Unnamed: 0.1,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,text,chunks,tokenized
Unnamed: 0,,,,,,,,,,,,


In [33]:
val_df[val_df['tokenized'].isnull()]

,Unnamed: 0.1,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,text,chunks,tokenized
Unnamed: 0,,,,,,,,,,,,


In [ ]:
train_df['author'].unique()

In [30]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV                

In [39]:
def lemmatize_text(tokenized_text):
    # Declaring Empty List to store the words that follow the rules for this step
    final_words = []
    # Initializing WordNetLemmatizer()
    word_lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(tokenized_text):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_final = word_lemmatized.lemmatize(word,tag_map[tag[0]])
            final_words.append(word_final)
    return str(final_words)
    # The final processed set of words for each iteration will be stored in 'text_final'

In [41]:
start = time.time()
with mp.Pool(11) as pool:
    lemmatized = pool.map(lemmatize_text, train_df['tokenized'])
end = time.time()
print(f'Took {end-start} seconds')
train_df['lemmatized'] = lemmatized

Took 265.2570126056671 seconds


In [42]:
import multiprocessing as mp
start = time.time()
with mp.Pool(11) as pool:
    lemmatized = pool.map(lemmatize_text, val_df['tokenized'])
end = time.time()
print(f'Took {end-start} seconds')
val_df['lemmatized'] = lemmatized

Took 34.088324546813965 seconds


In [43]:
import multiprocessing as mp
start = time.time()
with mp.Pool(11) as pool:
    lemmatized = pool.map(lemmatize_text, test_df['tokenized'])
end = time.time()
print(f'Took {end-start} seconds')
test_df['lemmatized'] = lemmatized

Took 34.880149364471436 seconds


In [ ]:
train_df.shape

In [47]:
Train_X, Train_Y = train_df['lemmatized'], train_df['author']
Test_X, Test_Y = test_df['lemmatized'], test_df['author']
Val_X, Val_Y = val_df['lemmatized'], val_df['author']

In [ ]:
# Train_X, _, Train_Y, _ = model_selection.train_test_split(train_df['lemmatized'], train_df['author'],test_size=0.3)
# Val_X, _, Val_Y, _ = model_selection.train_test_split(val_df['lemmatized'], val_df['author'],test_size=0.3)
# Test_X, _, Test_Y, _ = model_selection.train_test_split(test_df['lemmatized'], test_df['author'],test_size=0.3)


In [49]:
# Encoder = LabelEncoder()
# Train_Y_e = Encoder.fit_transform(Train_Y)
# Test_Y_e = Encoder.fit_transform(Test_Y)
# Val_Y_e = Encoder.fit_transform(Val_Y)


In [52]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(train_df['lemmatized'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
Val_X_Tfidf = Tfidf_vect.transform(Val_X)


In [513]:
def evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, res_file, model_description, preproc_desc):
    # Train and predict
    start = time.time()
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    end = time.time()

    print(f'Training and predicting took {end-start} seconds = {(end-start)/60} minutes')

    results={}
    for label, y_truth, y_pred in [('train', y_train, y_train_pred), 
                            ('validation', y_val, y_val_pred),
                            ('test', y_test, y_test_pred)]:
        # Metrics (set zero_division=0 to silence warnings)
        acc = accuracy_score(y_truth, y_pred)
        f1 = f1_score(y_truth, y_pred, average='weighted', zero_division=0)
        precision = precision_score(y_truth, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_truth, y_pred, average='weighted', zero_division=0)

        result_dict = {'accuracy': acc,
                       'precision': precision,
                       'recall' : recall,
                       'f1' : f1}
        results[label] = result_dict
        
        
    # Print performance
    print(f"Model: {model.__class__.__name__}")
    label_str=f'|{'':<15} ||  {'Accuracy':>15} | {'Precision':>15} | {'Recall':>15} | {'F1-Score':>15} |'
    print("-" * len(label_str))

    print(label_str)
    print("-" * len(label_str))

    for result_label, sub_res_dict in results.items():
        output_str = f'|{result_label:<15} || '
        
        for key, val in sub_res_dict.items():
            output_str += f' {val:15.4f} |'
        print(output_str)

    print("-" * len(label_str))

    new_res_df = results_to_df(model.__class__.__name__, model_description, preproc_desc, results)
    
    if os.path.exists(res_file):
        old_res_df = pd.read_csv(res_file)
        old_res_df.set_index(['model_type', 'description', 'preprocessing description', 'metric'], inplace=True)
    
        res_df = pd.concat([old_res_df, new_res_df])
        res_df.to_csv(res_file)
    else:
        new_res_df.to_csv(res_file)
        

    return model, results

In [515]:
def results_to_df(model_type, model_desc, preproc_desc, result_dict):
    res_df = pd.DataFrame.from_dict(result_dict)
    res_df['model_type'] = model_type
    res_df['description'] = model_desc
    res_df['preprocessing description'] = preproc_desc
    res_df.reset_index(inplace=True)
    res_df.rename({'index':'metric'}, axis=1, inplace=True)
    res_df.set_index(['model_type', 'description', 'preprocessing description', 'metric'], inplace=True)

    return res_df
    

# Support Vector Machines

In [525]:
outfile='/home/dean/Documents/gitRepos/gutenberg_corpus_analysis/SVM/results.csv'
preprocessing_description = '10 chunks, 10000 long, tf-idf 5000 max feat'

In [527]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
model = svm.SVC()
model_desc = 'default_settings'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

#train_class_rpt = classification_report(Train_Y, predictions_SVM_train)
#test_class_rpt = classification_report(Val_Y, predictions_SVM_val)


Training and predicting took 101.408536195755 seconds = 1.69014226992925 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9953 |          0.9956 |          0.9953 |          0.9954 |
|validation      ||           0.8167 |          0.8533 |          0.8167 |          0.8104 |
|test            ||           0.8208 |          0.8579 |          0.8208 |          0.8164 |
--------------------------------------------------------------------------------------------


In [528]:
# Classifier - Algorithm - SVM
model = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
model_desc = 'C=1, kernel Linear, deg 3, gamma auto'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 100.77923703193665 seconds = 1.6796539505322774 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9833 |          0.9846 |          0.9833 |          0.9835 |
|validation      ||           0.8542 |          0.8808 |          0.8542 |          0.8477 |
|test            ||           0.8500 |          0.8577 |          0.8500 |          0.8390 |
--------------------------------------------------------------------------------------------


In [548]:
# Classifier - Algorithm - SVM
model = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='scale')
model_desc='C=1, kernel Linear, deg 3 gamma scale'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 106.44879674911499 seconds = 1.7741466124852499 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9833 |          0.9846 |          0.9833 |          0.9835 |
|validation      ||           0.8542 |          0.8808 |          0.8542 |          0.8477 |
|test            ||           0.8500 |          0.8577 |          0.8500 |          0.8390 |
--------------------------------------------------------------------------------------------


In [549]:
# Classifier - Algorithm - SVM
model = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto')
model_desc='C=1, kernel rbf, deg 3 gamma auto'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 112.20486640930176 seconds = 1.870081106821696 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.8677 |          0.8902 |          0.8677 |          0.8641 |
|validation      ||           0.7375 |          0.7884 |          0.7375 |          0.7242 |
|test            ||           0.6833 |          0.7462 |          0.6833 |          0.6709 |
--------------------------------------------------------------------------------------------


In [550]:
# Classifier - Algorithm - SVM
model = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='scale')
model_desc='C=1, kernel rbf, deg 3 gamma scale'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 107.68566799163818 seconds = 1.7947611331939697 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9953 |          0.9956 |          0.9953 |          0.9954 |
|validation      ||           0.8167 |          0.8533 |          0.8167 |          0.8104 |
|test            ||           0.8208 |          0.8579 |          0.8208 |          0.8164 |
--------------------------------------------------------------------------------------------


In [551]:
# Classifier - Algorithm - SVM
model = svm.SVC(C=1.0, kernel='poly', degree=4, gamma='scale')
model_desc='C=1, kernel poly, deg 4 gamma scale'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 110.0511474609375 seconds = 1.834185791015625 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9995 |          0.9995 |          0.9995 |          0.9995 |
|validation      ||           0.7042 |          0.7820 |          0.7042 |          0.7116 |
|test            ||           0.7250 |          0.8052 |          0.7250 |          0.7347 |
--------------------------------------------------------------------------------------------


In [552]:
# Classifier - Algorithm - SVM
model = svm.SVC(C=1.0, kernel='poly', degree=2, gamma='scale')
model_desc='C=1, kernel poly, deg 2 gamma scale'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 107.77263116836548 seconds = 1.796210519472758 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9969 |          0.9971 |          0.9969 |          0.9969 |
|validation      ||           0.8208 |          0.8445 |          0.8208 |          0.8094 |
|test            ||           0.8167 |          0.8506 |          0.8167 |          0.8113 |
--------------------------------------------------------------------------------------------


In [553]:
# Classifier - Algorithm - SVM
model = svm.SVC(C=1.0, kernel='poly', degree=3, gamma='scale')
model_desc='C=1, kernel poly, deg 3 gamma scale'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 110.00115394592285 seconds = 1.8333525657653809 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9995 |          0.9995 |          0.9995 |          0.9995 |
|validation      ||           0.7625 |          0.8155 |          0.7625 |          0.7568 |
|test            ||           0.7542 |          0.8041 |          0.7542 |          0.7531 |
--------------------------------------------------------------------------------------------


In [554]:
# Classifier - Algorithm - SVM
model = svm.SVC(C=1.0, kernel='poly', degree=3, gamma='auto')
model_desc='C=1, kernel poly, deg 3 gamma auto'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 108.22327303886414 seconds = 1.8037212173144022 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9359 |          0.9532 |          0.9359 |          0.9345 |
|validation      ||           0.6417 |          0.7669 |          0.6417 |          0.6592 |
|test            ||           0.6125 |          0.7504 |          0.6125 |          0.6306 |
--------------------------------------------------------------------------------------------


In [555]:
# Classifier - Algorithm - SVM
model = svm.SVC(C=1.0, kernel='sigmoid', degree=3, gamma='scale')
model_desc='C=1, kernel sigmoid, deg 3 gamma scale'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 95.98268699645996 seconds = 1.5997114499409995 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9656 |          0.9683 |          0.9656 |          0.9660 |
|validation      ||           0.8500 |          0.8756 |          0.8500 |          0.8460 |
|test            ||           0.8250 |          0.8600 |          0.8250 |          0.8186 |
--------------------------------------------------------------------------------------------


In [556]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
model = svm.SVC(C=1.0, kernel='sigmoid', degree=3, gamma='auto')
model_desc='C=1, kernel sigmoid, deg 3 gamma auto'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 108.43932104110718 seconds = 1.8073220173517863 minutes
Model: SVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.8677 |          0.8902 |          0.8677 |          0.8641 |
|validation      ||           0.7375 |          0.7884 |          0.7375 |          0.7242 |
|test            ||           0.6833 |          0.7462 |          0.6833 |          0.6709 |
--------------------------------------------------------------------------------------------


In [558]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
model = svm.LinearSVC()
model_desc='default'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 5.753037929534912 seconds = 0.09588396549224854 minutes
Model: LinearSVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           1.0000 |          1.0000 |          1.0000 |          1.0000 |
|validation      ||           0.9167 |          0.9419 |          0.9167 |          0.9134 |
|test            ||           0.8958 |          0.8933 |          0.8958 |          0.8847 |
--------------------------------------------------------------------------------------------


In [559]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
model = svm.LinearSVC(C=10)
model_desc='C=10'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 7.698090314865112 seconds = 0.12830150524775188 minutes
Model: LinearSVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           1.0000 |          1.0000 |          1.0000 |          1.0000 |
|validation      ||           0.9250 |          0.9419 |          0.9250 |          0.9212 |
|test            ||           0.9167 |          0.9267 |          0.9167 |          0.9103 |
--------------------------------------------------------------------------------------------


In [560]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
model = svm.LinearSVC(C=10, multi_class='crammer_singer', max_iter=5000)
model_desc='C=10, crammer-singer, max_it 5000'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 17.52768325805664 seconds = 0.292128054300944 minutes
Model: LinearSVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           1.0000 |          1.0000 |          1.0000 |          1.0000 |
|validation      ||           0.9208 |          0.9427 |          0.9208 |          0.9174 |
|test            ||           0.9125 |          0.9235 |          0.9125 |          0.9076 |
--------------------------------------------------------------------------------------------


/home/dean/anaconda3/envs/school-env/lib/python3.12/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [561]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
model = svm.NuSVC()
model_desc='default'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 102.72420239448547 seconds = 1.7120700399080913 minutes
Model: NuSVC
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9948 |          0.9953 |          0.9948 |          0.9949 |
|validation      ||           0.8375 |          0.8539 |          0.8375 |          0.8270 |
|test            ||           0.8250 |          0.8567 |          0.8250 |          0.8197 |
--------------------------------------------------------------------------------------------


# Naive Bayes

In [562]:
# fit the training dataset on the NB classifier
model = naive_bayes.MultinomialNB()
model_desc='default'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 0.18079328536987305 seconds = 0.0030132214228312175 minutes
Model: MultinomialNB
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.9005 |          0.9287 |          0.9005 |          0.9030 |
|validation      ||           0.6708 |          0.7280 |          0.6708 |          0.6462 |
|test            ||           0.7042 |          0.7653 |          0.7042 |          0.6843 |
--------------------------------------------------------------------------------------------


# Neural Networks

## Multi Layer Perceptron

In [563]:
model = neural_network.MLPClassifier()
model_desc='default'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 51.74481916427612 seconds = 0.8624136527379354 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           1.0000 |          1.0000 |          1.0000 |          1.0000 |
|validation      ||           0.9125 |          0.9321 |          0.9125 |          0.9088 |
|test            ||           0.9167 |          0.9279 |          0.9167 |          0.9135 |
--------------------------------------------------------------------------------------------


In [564]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(200,), max_iter=1000)
model_desc='hidden_layer_size 200, max it 1000'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 87.38759231567383 seconds = 1.4564598719278972 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           1.0000 |          1.0000 |          1.0000 |          1.0000 |
|validation      ||           0.9208 |          0.9396 |          0.9208 |          0.9186 |
|test            ||           0.9125 |          0.9173 |          0.9125 |          0.9062 |
--------------------------------------------------------------------------------------------


In [565]:
model = neural_network.MLPClassifier(activation='logistic', max_iter=500)
model_desc='logistic act, 500 iter'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 99.63221025466919 seconds = 1.6605368375778198 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           1.0000 |          1.0000 |          1.0000 |          1.0000 |
|validation      ||           0.9167 |          0.9344 |          0.9167 |          0.9151 |
|test            ||           0.9042 |          0.9162 |          0.9042 |          0.8997 |
--------------------------------------------------------------------------------------------


In [596]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(200,), activation='logistic', max_iter=500)
model_desc='hidden layer size 200, logistic act, 500 iter'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 273.6840138435364 seconds = 4.5614002307256065 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           1.0000 |          1.0000 |          1.0000 |          1.0000 |
|validation      ||           0.9083 |          0.9356 |          0.9083 |          0.9081 |
|test            ||           0.9250 |          0.9350 |          0.9250 |          0.9187 |
--------------------------------------------------------------------------------------------


In [612]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(200,), activation='logistic', max_iter=500, learning_rate='invscaling', solver='sgd')
model_desc='hidden layer size 200, logistic act, 500 iter, sgd-invscaling'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 8.986948013305664 seconds = 0.14978246688842772 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.0125 |          0.0002 |          0.0125 |          0.0003 |
|validation      ||           0.0125 |          0.0002 |          0.0125 |          0.0003 |
|test            ||           0.0125 |          0.0002 |          0.0125 |          0.0003 |
--------------------------------------------------------------------------------------------


In [604]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(500,), activation='relu', max_iter=500, learning_rate='invscaling', solver='sgd')
model_desc='hidden layer size 500, relu act, 500 iter, sgd-invscaling'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 21.707059860229492 seconds = 0.36178433100382484 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.0109 |          0.0005 |          0.0109 |          0.0009 |
|validation      ||           0.0125 |          0.0005 |          0.0125 |          0.0010 |
|test            ||           0.0083 |          0.0002 |          0.0083 |          0.0004 |
--------------------------------------------------------------------------------------------


In [617]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(500,), activation='relu', max_iter=500, learning_rate='constant', solver='sgd')
model_desc='hidden layer size 500, relu act, 500 iter, sgd-constant'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

/home/dean/anaconda3/envs/school-env/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and predicting took 851.5928704738617 seconds = 14.193214507897695 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.4568 |          0.7121 |          0.4568 |          0.4622 |
|validation      ||           0.3250 |          0.3742 |          0.3250 |          0.3133 |
|test            ||           0.3375 |          0.3766 |          0.3375 |          0.3095 |
--------------------------------------------------------------------------------------------


In [614]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(500,), activation='relu', max_iter=500, solver='adam')
model_desc='hidden layer size 500, relu act, 500 iter, adam'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 185.4526436328888 seconds = 3.0908773938814798 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           1.0000 |          1.0000 |          1.0000 |          1.0000 |
|validation      ||           0.9292 |          0.9440 |          0.9292 |          0.9265 |
|test            ||           0.9167 |          0.9225 |          0.9167 |          0.9103 |
--------------------------------------------------------------------------------------------


In [609]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(500,200, 100, ), activation='relu', max_iter=500, learning_rate='invscaling', solver='sgd')
model_desc='hidden layer size 500-200-100, relu act, 500 iter, sgd-invscaling'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 23.782085180282593 seconds = 0.3963680863380432 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.0146 |          0.0004 |          0.0146 |          0.0007 |
|validation      ||           0.0083 |          0.0002 |          0.0083 |          0.0004 |
|test            ||           0.0083 |          0.0002 |          0.0083 |          0.0005 |
--------------------------------------------------------------------------------------------


In [ ]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(500,200, 100, ), activation='relu', max_iter=500, learning_rate='constant', solver='sgd')
model_desc='hidden layer size 500-200-100, relu act, 500 iter, sgd-constant'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

In [615]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(500,200, 100, ), activation='relu', max_iter=500, solver='adam')
model_desc='hidden layer size 500-200-100, relu act, 500 iter, adam'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 127.03732872009277 seconds = 2.1172888120015463 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           1.0000 |          1.0000 |          1.0000 |          1.0000 |
|validation      ||           0.8833 |          0.9142 |          0.8833 |          0.8776 |
|test            ||           0.8375 |          0.8545 |          0.8375 |          0.8272 |
--------------------------------------------------------------------------------------------


In [606]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(200,100, ), activation='relu', max_iter=500, learning_rate='invscaling', solver='sgd')
model_desc='hidden layer size 200-100, relu act, 500 iter, sgd-invscaling'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 8.672466516494751 seconds = 0.14454110860824584 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.0135 |          0.0008 |          0.0135 |          0.0013 |
|validation      ||           0.0167 |          0.0010 |          0.0167 |          0.0017 |
|test            ||           0.0125 |          0.0002 |          0.0125 |          0.0003 |
--------------------------------------------------------------------------------------------


In [616]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(200,100, ), activation='relu', max_iter=500, solver='adam')
model_desc='hidden layer size 200-100, relu act, 500 iter, adam'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 67.76165580749512 seconds = 1.1293609301249186 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           1.0000 |          1.0000 |          1.0000 |          1.0000 |
|validation      ||           0.8958 |          0.9117 |          0.8958 |          0.8930 |
|test            ||           0.9000 |          0.9237 |          0.9000 |          0.8950 |
--------------------------------------------------------------------------------------------


In [607]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(200,200, ), activation='relu', max_iter=500, learning_rate='invscaling', solver='sgd')
model_desc='hidden layer size 200-200, relu act, 500 iter, sgd-invscaling'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

Training and predicting took 8.011568307876587 seconds = 0.13352613846460978 minutes
Model: MLPClassifier
--------------------------------------------------------------------------------------------
|                ||         Accuracy |       Precision |          Recall |        F1-Score |
--------------------------------------------------------------------------------------------
|train           ||           0.0125 |          0.0002 |          0.0125 |          0.0003 |
|validation      ||           0.0125 |          0.0002 |          0.0125 |          0.0003 |
|test            ||           0.0125 |          0.0002 |          0.0125 |          0.0003 |
--------------------------------------------------------------------------------------------


In [ ]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(200,200, ), activation='relu', max_iter=500, learning_rate='constant', solver='sgd')
model_desc='hidden layer size 200-200, relu act, 500 iter, sgd-constant'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

In [ ]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(200,200, ), activation='relu', max_iter=500, solver='adam')
model_desc='hidden layer size 200-200, relu act, 500 iter, adam'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

In [ ]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(100,50, ), activation='relu', max_iter=500, solver='adam')
model_desc='hidden layer size 100-50, relu act, 500 iter, adam'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

In [ ]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(100,100, ), activation='relu', max_iter=500, solver='adam')
model_desc='hidden layer size 100-100, relu act, 500 iter, adam'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)

In [ ]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(100,100, 100,), activation='relu', max_iter=500, solver='adam')
model_desc='hidden layer size 100-100-100, relu act, 500 iter, adam'
model, result_dict = evaluate_model(model, Train_X_Tfidf, Train_Y, Val_X_Tfidf, Val_Y, Test_X_Tfidf, Test_Y, 
                                    outfile, model_desc, preprocessing_description)